#   CNN(卷积神经网络)
## 一、全连接层的缺点
在mlp章节，我们对Fashion-MNIST图像集进行分类预测，每个图像转化为tensor形式都满足 $ shape\in R^{a*b*c}$ (其中a*b为图像的几何大小，c为channel数，代表RGB颜色)，我们使用全连接层结合softmax回归进行预测的第一步便是将三个维度拉伸成一维向量进行处理，此时经过Flatten层后第一个weight的 $ shape\in R^{784*256}$ 当图像的三维度之和非常大时全连接层将会产生巨大的weight数量，消耗更多的gpu资源
## 二、如何优化
* 每一个权重代表数据集一部分数据拟合的结果（这意味着不需要为每一个维度都设置一个weight）
* weight在处理单一batch时不再是一个一维向量，而是满足与数据集相似的  $ shape\in R^{a*b*c}$
## 三、互相关计算
考虑一个二维图片的计算：

![avatar](https://zh.d2l.ai/_images/correlation.svg)

* 核函数（kernel）将依照从左到右再从上到下的方式遍历数据集，每次步长（stride）为 1 ，并以此按元素进行乘法运算形成weight。

\begin{split}0\times0+1\times1+3\times2+4\times3=19,\\
1\times0+2\times1+4\times2+5\times3=25,\\
3\times0+4\times1+6\times2+7\times3=37,\\
4\times0+5\times1+7\times2+8\times3=4.
\end{split}

* 其中输出层的shape满足（stride==1时）：

$ shape=(n_h-k_h+1) \times (n_w-k_w+1) $

* feature map的含义：在本例中weight是一个按层序元素value增大的特征向量，而在经过卷积运算之后，feature map也是一个按层序元素value增大的矩阵，也就是说**该feature map经过卷积后拥有了该weight的特征。**



In [1]:
# 模拟互相关运算
import torch
from torch import nn

def corr2d(X,k):
    Yx=X.shape[0]-k.shape[0]+1
    Yy=X.shape[1]-k.shape[1]+1
    Y=torch.zeros((Yx,Yy))
    for i in range(Yx):
        for j in range(Yy):
            Y[i,j]=(X[i:i+k.shape[0],j:j+k.shape[1]]*k).sum()
    return Y
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

e:\coding_setting\Miniconda\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[19., 25.],
        [37., 43.]])

## 四、卷积层的构建

In [2]:

class conv2d(nn.Module):
    def __init__(self,kernel_size):
        super().__init__()
        weight=nn.Parameter(torch.randn(kernel_size))
        bias=nn.Parameter(torch.ones(1)) #之后调用广播机制完成加法

    def forward(self,x):
        return corr2d(x,self.weight)+self.bias

## 五、卷积层的初步使用
*学习由X生成Y的卷积核。*

*通过找到像素变化的位置，来检测图像中不同颜色的边缘。*

In [3]:
import torch
from torch import nn

#构造图像区域的二维矩阵，其中黑色为1，白色为0
X=torch.zeros((6,8))
X[:,2:6]=1
X

tensor([[0., 0., 1., 1., 1., 1., 0., 0.],
        [0., 0., 1., 1., 1., 1., 0., 0.],
        [0., 0., 1., 1., 1., 1., 0., 0.],
        [0., 0., 1., 1., 1., 1., 0., 0.],
        [0., 0., 1., 1., 1., 1., 0., 0.],
        [0., 0., 1., 1., 1., 1., 0., 0.]])

In [4]:

true_k=torch.tensor([[1,-1]])
y=corr2d(X,true_k)
y
#0代表不是边界，1代表从黑到白，0代表从白到黑

tensor([[ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.]])

In [5]:
conv2d=nn.Conv2d(1,1,kernel_size=(1,2),bias=False)

def train_conv2d(X,y,lr,batch,channals,times):
    X=X.reshape((batch,channals,6,8))
    y=y.reshape((batch,channals,6,7))
    for i in range(times):
        y_hat=conv2d(X)
        loss = (y_hat - y) ** 2
        conv2d.zero_grad()
        loss.sum().backward()
        conv2d.weight.data[:]-=lr*conv2d.weight.grad
        print(f'第{i+1}次操作,loss为{loss.sum()}',end='\n')

In [6]:
train_conv2d(X,y,0.01,1,1,10)
conv2d.weight.data

第1次操作,loss为10.286530494689941
第2次操作,loss为7.937241077423096
第3次操作,loss为6.145866394042969
第4次操作,loss为4.759340286254883
第5次操作,loss为3.6856327056884766
第6次操作,loss为2.854154348373413
第7次操作,loss为2.210257053375244
第8次操作,loss为1.711622953414917
第9次操作,loss为1.3254806995391846
第10次操作,loss为1.0264525413513184


tensor([[[[ 0.7426, -0.7426]]]])

## 六、卷积层的参数设置
# 1.paddings：

![avatar](https://zh.d2l.ai/_images/conv-pad.svg)

$$
shape \in R^{(n_h-k_h+p_h+1)*(n_w-k_w+p_w+1)} (p_h为row padding，p_w为column padding)
$$

一般我们会让输入层与输出层具有一样的shape，因此我们通常设置kernel的height和width都为奇数，这样我们就可以在顶部和底部添加相同数量的paddings

# 2.stride：

$$
shape \in R^{((n_h-k_h+p_h+s_h)/s_h)*((n_w-k_w+p_w++s_p)/s_p)} (s_h为vertical stride，s_p为horizon stride)
$$


## 七、输入输出通道多样性
### 1.多输入通道
当输入图像为彩色图象时，其矩阵表示为三维矩阵，多出来的维度代表了颜色channel，因此卷积核也应该是三维卷积核，并且进行三维的互相关运算

![avatar](https://zh.d2l.ai/_images/conv-multi-in.svg)



In [8]:
def multi_corr2d_in(X,k):
    return sum(corr2d(X0,k0) for X0,k0 in zip(X,k))
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

multi_corr2d_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

### 2.多输出通道
前面的运算最后得出的仍然是二维矩阵，但实际上我们可以有多个三维卷积核，每个核生成一个输出通道，保持多通道的输出使每个通道对应不同的特征，也即满足：

$$
X \in R^{c_i*n_h*n_w} \\
k \in R^{c_o*c_i*k_h*k_w} \\
Y \in R^{c_o*m_h*m_w}
$$

In [ ]:
def multi_corr2d_in_out(X,k):
    return torch.stack([multi_corr2d_in(X, k0) for k0 in k],dim=0)

## 八、1*1卷积核

![avatar](https://zh.d2l.ai/_images/conv-1x1.svg)

* 1*1卷积核实际上实现了一个全连接层
* 1*1卷积层通常用于调整网络层的通道数量和控制模型复杂性。
*  卷积通道数取决于该层的卷积层，也就是是一个超参数
* 多输出通道卷积本质就是每一个三维卷积都对X进行一次互相关运算

## 九、池化层（pooring）
在使用卷积层时，我们希望在每次卷积后都能够形成一个特征，并且其receptive field越大，在最后一层应该包含所有的输入图像。使用池化层，它具有双重目的：降低卷积层对位置的敏感性，同时降低对空间降采样表示的敏感性。
### 1.max pooring
字面意思，对每个区域进行求最大值操作
![avatar](https://zh.d2l.ai/_images/pooling.svg)
### 2.average pooring
字面意思，对每个区域进行求平均值操作

In [10]:
import torch
from torch import nn
def pool2d(X,pool_size,mode):
    lp,wp=pool_size
    lx,wx=X.shape
    ly,wy=lx-lp+1,wx-wp+1
    y=torch.zeros((ly,wy))
    for i in range(ly):
        for j in range(wy):
            if(mode=='max'):
                y[i][j]=torch.max(X[i:i+wp,j:j+lp])
            else:
                y[i][j]=torch.mean(X[i:i+wp,j:j+lp])
    return y
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2),'max')

tensor([[4., 5.],
        [7., 8.]])

### 3.多通道情况
与卷积层不同的是，池化层在每个输入通道上单独运算，也就是说池化层输出和输入的通道数相同、
### 4.padding stride
与卷积层的计算类似